In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import tensorflow as tf

In [ ]:

url = 'https://raw.githubusercontent.com/krishnaik06/AQI-Project/master/Data/Real-Data/Real_Combine.csv'

df = pd.read_csv(url, error_bad_lines=False)


In [ ]:
df.head()

In [ ]:
df=df.dropna()
df

In [ ]:

X=df.iloc[:,:-1] ## independent features
y=df.iloc[:,-1] ## dependent features

from sklearn.preprocessing import StandardScaler
sc_x = StandardScaler()
sc_y = StandardScaler()
# transform data
X= sc_x.fit_transform(X)
y=sc_y.fit_transform(y.values.reshape(-1,1))

print(X)



# Splitting into train and test

In [ ]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=0)


# Making ANN layers and complilation

In [ ]:

from tensorflow.keras import regularizers
from tensorflow.keras.callbacks import EarlyStopping





model = tf.keras.Sequential()
model.add(tf.keras.layers.Dense(units=8, kernel_initializer='normal',activation='relu',input_dim=8)

model.add(tf.keras.layers.Dense(units=6, kernel_initializer='normal',activation='relu'))

model.add(tf.keras.layers.Dense(units=6, kernel_initializer='normal',activation='relu'))

model.add(tf.keras.layers.Dense(units=6, kernel_initializer='normal',activation='relu'))

model.add(tf.keras.layers.Dense(units=1, kernel_initializer='normal',activation='linear'))

# Compile the network :
opt=tf.keras.optimizers.Adam(
    learning_rate=0.001,
    beta_1=0.9,
    beta_2=0.999)
model.compile(loss='mean_absolute_error', optimizer=opt, metrics=['mean_absolute_error'])

monitor = EarlyStopping(monitor='val_loss', min_delta=1e-3, patience=15, 
        verbose=1, mode='auto', restore_best_weights=True)



model.summary()



# Fitting ANN model

In [ ]:
# Fitting the ANN to the Training set
model_history=model.fit(X_train, y_train,validation_split=0.33, batch_size = 10,callbacks=[monitor], epochs = 400)

In [ ]:
print(model_history.history.keys())

In [ ]:
plt.plot(model_history.history['val_loss'])
plt.plot(model_history.history['loss'])

plt.title('model loss')
plt.ylabel('loss')
plt.xlabel('epoch')
plt.legend(['training','validation'], loc='upper right')
plt.show()


# Model Evaluation


Interpreting the coefficients:

* Holding all other features fixed, a 1 unit increase in T is associated with an decrease of 2.690 in AQI PM2.5 .
* Holding all other features fixed, a 1 unit increase in TM is associated with an increase of 0.46 in AQI PM 2.5 .

In [ ]:

prediction=model.predict(X_test)

plt.scatter(sc_y.inverse_transform(y_test),sc_y.inverse_transform(prediction))


In [ ]:
from sklearn import metrics
print('MSE:', metrics.mean_squared_error(sc_y.inverse_transform(y_test), sc_y.inverse_transform(prediction)))

In [ ]:
model.save_weights('ANN_air_quality.h5')


